In [1]:
import numpy as np

# Run xml on Python

In [4]:
# link_id = 511 (node: 501 -> 4) The next link_id = 5 (node: 4 -> 5)
# demand_511: 1800
# length_511: 1000
# demand_5: 1000 (after 1800 seconds)
# length_5: 228
# state_space['511'] = demand_511 / length_511 = 1.8
# state_space['5'] = demand_5 / length_5


#<roadparam name="Freeway" id="0" jam_density="120" speed="120" capacity="2000"/>
#<roadparam name="Ramp" id="1" jam_density="120" speed="80" capacity="900"/>

den_jam = 120
den_none = 0
ramp_1_length = 10

# ramp per 60 seconds moves forward 80 / (3600/60) = 1.33 miles, 1.33 * ramp_1_length


# when there is only "511": state_space(density) can only be den_none ~ den_jam


print(state_space)



array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110])

In [19]:
#Initialize Q-table with all zeros
list_link_ids = ['27','28','29','30','31','32','33']
list_ramp_ids = ['511','911','1011','1511'] # inlink for '27','29','30','33'

# Version A (when we have OTM functions):

# list_link_den     = otm.get_density(list_link_ids)
# list_ramp_speed   = otm.get_speed(list_ramp_ids)
# list_ramp_flow    = otm.get_flow(list_ramp_ids)
# list_link_den_jam = otm.get_jam_density(list_link_ids)

# Version B (when we don't have OTM functions):

list_link_den     = [30,50,60,70,90,80,70]
list_ramp_speed   = [80,80,80,80]
list_ramp_flow    = [800,800,800,800]
list_link_den_jam = [120,120,120,120,120,120,120]
den_none = 0


possible_den    = np.arange(den_none, max(list_link_den_jam), 10)
possible_action = ['full_red', 'full_green']

state_space = np.arange( 1, len(possible_den)*len(list_link_ids)+1, 1)
act_space = np.arange( 1, len(possible_action)*len(list_ramp_ids)+1, 1)

print(state_space)
print(act_space)

#Q    = np.zeros([len(state_space),len(act_space)])
#Qnew = np.ones([len(state_space),len(act_space)])


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84]
[1 2 3 4 5 6 7 8]


In [7]:
# otm.run()
list_link_ids = ['511','911','1011','1511']
# list_link_den = otm.get_density(list_link_ids)
list_link_den = [30,50,90,80]
# list_den_cri = otm.get_critical_density(list_link_ids)
list_den_cri = [60,60,60,60]

list_r = []
def reward(list1, list2, tolerance):
    for i in range(len(list1)):
        if abs( list1[i] - list2[i] ) <= tolerance:
            r = 1
        elif list1[i] - list2[i] <= 0:
            r = 0.5
        else:
            r = -1
        list_r.append(r)
    
    return min(list_r)
    
reward = reward(list_link_den, list_den_cri, 10)
print(reward)

-1


In [19]:
state_space = [10,20,30,40,50,60,70,80,90,100] # density (#car/mile)
act_space = [-10,10]

den_cri = 60

In [20]:

# Set learning parameters
learning = 0.8
discount = 0.95
num_episodes = 2000
max_gap = 0.01 

In [21]:
def step(s,a):
    global sp
    den = state_space[s] + act_space[a]
    for i in range(len(state_space)):
        
        if i == 0:
            if 0 <= den < state_space[i]:
                sp = i
        elif state_space[i-1] <= den < state_space[i]:
            sp = i
            
    # make it to be continuous??
    
    if den_cri - den >= 0:
        #r = 1/(den_cri - den +1)
        #r = 1/(den - den_cri+1)**2 
        r = 1/(den_cri - den +1)
    elif den_cri - den <= 0:
        #r = 1/(den - den_cri +1)
        #r = 1/(den - den_cri+1)**2 
        r = den_cri - den
    return sp, r

In [22]:
r_list = []
for i in range(num_episodes):
    
    # Reset the environment
    s     = 0
    r_sum = 0
    j     = 0
    
    while j < 5000:
            
        # Q-table iteration
            
        j += 1
        noise = np.random.randn(1,len(act_space))*(1./(i+1))
        a = np.argmax(Q[s,:] + noise)
        s1,r = step(s,a)
        
        Q[s,a] = Q[s,a] + learning * (r + discount * np.max( Q[s1,:] ) - Q[s,a] )
        
        r_sum += r
        s = s1
        
        gap = abs(Qnew[s,a] - Q[s,a])
        
        if gap < max_gap:
            break
        
        Qnew[s,a] = Q[s,a]
        
    r_list.append(r_sum)

In [23]:
print(Q) # continuous, not differentiable (symmetric)

[[ 5.55366364e-02  1.79171583e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 5.23670151e-02  1.88344927e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.97768135e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 1.97663346e+01 -1.60000000e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [-3.99767560e+02 -4.13292272e+02]
 [ 0.00000000e+00  0.00000000e+00]]


In [24]:
print(Q) # continuous, differentiable  (symmetric)

[[ 5.55366364e-02  1.79171583e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 5.23670151e-02  1.88344927e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.97768135e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 1.97663346e+01 -1.60000000e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [-3.99767560e+02 -4.13292272e+02]
 [ 0.00000000e+00  0.00000000e+00]]


In [25]:
print(Q) #  not-continous (asymmetric)
print(np.argmax(Q[:,1]))

[[ 5.55366364e-02  1.79171583e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 5.23670151e-02  1.88344927e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.97768135e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [ 1.97663346e+01 -1.60000000e+01]
 [ 0.00000000e+00  0.00000000e+00]
 [-3.99767560e+02 -4.13292272e+02]
 [ 0.00000000e+00  0.00000000e+00]]
4


In [26]:
print(r_list)

[-4398.887207021953, 110.09639953542393, 1.0884027341444376, 1.0720092915214867, 1.0720092915214867, 1.0720092915214867, 1.0720092915214867, 1.0720092915214867, 0.07200929152148663, 0.07200929152148663, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024390243902439025, 0.024